In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sklearn
from sklearn import metrics, tree
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_predict
from sklearn.tree import DecisionTreeClassifier, plot_tree

In [ ]:
xls = pd.ExcelFile('Dataset - LBP RA.xlsx')
dataframe = pd.read_excel(xls, 'Training Dataset')

In [ ]:
print(dataframe.info())

In [ ]:
column_types = {
    "Treatment": {1: "yes", 0: "no"},
    "Fever": {1: "yes", 0: "no"},
    "Duration_of_pain": {
        1: "0-1 weeks",
        2: "1-2 weeks",
        3: "3-4 weeks",
        4: "4-5 weeks",
        5: "6-8 weeks",
        6: "9-11 weeks",
        7: "3-6 months",
        8: "6-9 months",
        9: "9-12 months",
        10: "more than 1 year",
    },
    "Sick_leave": {1: "yes", 0: "no"},
    "Earlier_hospitalization": {1: "yes", 0: "no"},
    "Workoverload": {1: "yes", 0: "no"},
    "Familiy_history": {1: "yes", 0: "no"},
    "Depression": {1: "yes", 0: "no"},
    "Extremely_nervous": {
        0: "Not at all",
        1: "1",
        2: "2",
        3: "3",
        4: "4",
        5: "5",
        6: "6",
        7: "7",
        8: "8",
        9: "9",
        10: "Very tense or anxious",
    },
    "Stress": {
        0: "Not at all",
        1: "1",
        2: "2",
        3: "3",
        4: "4",
        5: "5",
        6: "6",
        7: "7",
        8: "8",
        9: "9",
        10: "completely satisfied",
    },
    "Relationship_with_colleagues": {
        0: "Not risk",
        1: "1",
        2: "2",
        3: "3",
        4: "4",
        5: "5",
        6: "6",
        7: "7",
        8: "8",
        9: "9",
        10: "Very high risk",
    },
    "Irrational_thoughts_risk_lasting": {
        0: "fully disagree",
        1: "1",
        2: "2",
        3: "3",
        4: "4",
        5: "5",
        6: "6",
        7: "7",
        8: "8",
        9: "9",
        10: "fully agree",
    },
    "Irrational_thoughts_work": {
        1: "fully disagree",
        2: "1",
        3: "2",
        4: "3",
        5: "4",
        6: "5",
        7: "6",
        8: "7",
        9: "8",
        10: "9",
        11: "fully agree",
    },
    "Coping_strategy": {
        1: "fully disagree",
        2: "1",
        3: "2",
        4: "3",
        5: "4",
        6: "5",
        7: "6",
        8: "7",
        9: "8",
        10: "9",
        11: "fully agree",
    },
    "Kinesiophobia_physical_exercise": {
        1: "fully disagree",
        2: "1",
        3: "2",
        4: "3",
        5: "4",
        6: "5",
        7: "6",
        8: "7",
        9: "8",
        10: "9",
        11: "Can reduce the pain completely",
    },
    "Kinesiophobia_pain_stop": {
        1: "fully disagree",
        2: "1",
        3: "2",
        4: "3",
        5: "4",
        6: "5",
        7: "6",
        8: "7",
        9: "8",
           10: "9",
        11: "Can reduce the pain completely",
    },
    "Age": {
        "0-19": "20-29",
        "20-29": "20-29",
        "30-39": "30-39",
        "40-49": "40-49",
        "50-59": "50-59",
        "60-69": "60-69",
        ">=80": ">=80",
    },
    "Uses_analgesics": {1: "yes", 0: "no"},
    "Uses_corticosteroids": {1: "yes", 0: "no"},
    "Serious_disease": {1: "yes", 0: "no"},
    "Neurogenic_signals": {1: "yes", 0: "no"},
    "Continuous_pain": {1: "yes", 0: "no"},
    "Decreased_mobility": {1: "yes", 0: "no"},
    "Nocturnal_pain": {1: "yes", 0: "no"},
    "Weightloss_per_year": {
        1: "decrease of weight",
        2: "increase of weight",
        3: "stable",
        4: "fluctuating",
    },
    "Loss_muscle_strength": {
        1: "None",
        2: "None",
        3: "None",
        4: "None",
        5: "Unbearable",
    },
    "Trauma": {1: "None", 2: "None", 3: "None", 4: "Unbearable", 5: "Unbearable"},
    "Failure_symptoms": {1: "yes", 0: "no"},
    "Incoordination": {
        1: "None",
        2: "None",
        3: "Unbearable",
        4: "Unbearable",
        5: "Unbearable",
    },
    "neck_pain_intensity": {
        1: "None",
        2: "None",
        3: "Unbearable",
        4: "Unbearable",
        5: "Unbearable",
    },
    "low_back_pain_intensity": {
        1: "None",
        2: "None",
        3: "Unbearable",
        4: "Unbearable",
        5: "Unbearable",
    },
    "arm_left_pain_intensity": {
        1: "None",
        2: "None",
        3: "Unbearable",
        4: "Unbearable",
        5: "Unbearable",
    },
    "arm_right_pain_intensity": {
        1: "None",
        2: "None",
        3: "Unbearable",
        4: "Unbearable",
        5: "Unbearable",
    },
    "leg_left_pain_intensity": {
        1: "None",
        2: "None",
        3: "Unbearable",
        4: "Unbearable",
        5: "Unbearable",
    },
    "leg_right_pain_intensity": {
        1: "None",
        2: "None",
        3: "Unbearable",
        4: "Unbearable",
        5: "Unbearable",
    },
    "working_ability": {
        1: "None",
        2: "None",
        3: "Unbearable",
        4: "Unbearable",
        5: "Unbearable",
        6: "Unbearable",
        7: "Unbearable",
        8: "Unbearable",
        9: "Unbearable",
        10: "Unbearable",
    },
    "Paidwork": {1: "yes", 0: "no"},
}

In [ ]:
X = dataframe[['Fever', 'Duration_of_pain']]
y = dataframe['Treatment'] 
X = pd.get_dummies(X, drop_first=True)

# Decision Tree Model
tree_model = DecisionTreeClassifier(max_depth=3)
tree_predicted = cross_val_predict(tree_model, X, y, cv=10)
tree_model.fit(X, y)

# Evaluation
print("Simple k=10 K fold CV")
print("Decision Tree Model:")
print(classification_report(y, tree_predicted))

In [ ]:
plt.figure(figsize=(90, 60))
plot_tree(tree_model, feature_names=X.columns, filled=True, rounded=True)
plt.show()